In [54]:
import numpy as np

from matrix_generator import generate_matrix
import subprocess
import re
from pathlib import Path
import pandas as pd

In [2]:
matrix_sizes = [0x200, 0x400, 0x600, 0x800, 0x1000, 0x1200, 0x1400, 0x1600, 0x1800, 0x2000]
patterns = ["checkerboard", "blockdiagonal", "random"]

In [3]:
#define test matrix sizes

matrix_files = []

# generate matrix input sizes
for matrix_size in matrix_sizes:
    for pattern in patterns:
        matrix_name = f"tests/MatrixA_{matrix_size}_{pattern}.mat"
        matrix_files.append((matrix_name, matrix_size, pattern))
        # generate_matrix(output=matrix_name,
        #                 num_cols=matrix_size,
        #                 num_rows=matrix_size,
        #                 sparsity=0.0,
        #                 dtype='float16',
        #                 heatmap=False,
        #                 pattern=pattern,
        #                 blocksize=16)
        print("Created " + matrix_name)

Created tests/MatrixA_512_checkerboard.mat
Created tests/MatrixA_512_blockdiagonal.mat
Created tests/MatrixA_512_random.mat
Created tests/MatrixA_1024_checkerboard.mat
Created tests/MatrixA_1024_blockdiagonal.mat
Created tests/MatrixA_1024_random.mat
Created tests/MatrixA_1536_checkerboard.mat
Created tests/MatrixA_1536_blockdiagonal.mat
Created tests/MatrixA_1536_random.mat
Created tests/MatrixA_2048_checkerboard.mat
Created tests/MatrixA_2048_blockdiagonal.mat
Created tests/MatrixA_2048_random.mat
Created tests/MatrixA_4096_checkerboard.mat
Created tests/MatrixA_4096_blockdiagonal.mat
Created tests/MatrixA_4096_random.mat
Created tests/MatrixA_4608_checkerboard.mat
Created tests/MatrixA_4608_blockdiagonal.mat
Created tests/MatrixA_4608_random.mat
Created tests/MatrixA_5120_checkerboard.mat
Created tests/MatrixA_5120_blockdiagonal.mat
Created tests/MatrixA_5120_random.mat
Created tests/MatrixA_5632_checkerboard.mat
Created tests/MatrixA_5632_blockdiagonal.mat
Created tests/MatrixA_563

In [ ]:
# test the code
for i in range(0, len(matrix_files), 3):
    for run in range(30):
        print(f"Run {run}")
        print("Running " + matrix_files[i][0])
        result = subprocess.run(["./cmake-build-debug/matrix_multiplication",
                                 matrix_files[i][0], matrix_files[i+2][0]], stdout=subprocess.PIPE)
        print("Done running " + matrix_files[i][0])
        with open(f"tests/MatrixC_{matrix_files[i][1]}_{matrix_files[i][2]}_run{run}.res", "w") as f:
            f.write(result.stdout.decode('utf-8'))

        print("Running " + matrix_files[i+1][0])
        result = subprocess.run(["./cmake-build-debug/matrix_multiplication",
                                 matrix_files[i+1][0], matrix_files[i+2][0]], stdout=subprocess.PIPE)
        print("Done running " + matrix_files[i+1][0])
        with (open(f"tests/MatrixC_{matrix_files[i+1][1]}_{matrix_files[i+1][2]}_run{run}.res", "w")
              as f):
            f.write(result.stdout.decode('utf-8'))

Run 0
Running tests/MatrixA_512_checkerboard.mat
Done running tests/MatrixA_512_checkerboard.mat
Running tests/MatrixA_512_blockdiagonal.mat
Done running tests/MatrixA_512_blockdiagonal.mat
Run 1
Running tests/MatrixA_512_checkerboard.mat
Done running tests/MatrixA_512_checkerboard.mat
Running tests/MatrixA_512_blockdiagonal.mat
Done running tests/MatrixA_512_blockdiagonal.mat
Run 2
Running tests/MatrixA_512_checkerboard.mat
Done running tests/MatrixA_512_checkerboard.mat
Running tests/MatrixA_512_blockdiagonal.mat
Done running tests/MatrixA_512_blockdiagonal.mat
Run 3
Running tests/MatrixA_512_checkerboard.mat
Done running tests/MatrixA_512_checkerboard.mat
Running tests/MatrixA_512_blockdiagonal.mat
Done running tests/MatrixA_512_blockdiagonal.mat
Run 4
Running tests/MatrixA_512_checkerboard.mat
Done running tests/MatrixA_512_checkerboard.mat
Running tests/MatrixA_512_blockdiagonal.mat
Done running tests/MatrixA_512_blockdiagonal.mat
Run 5
Running tests/MatrixA_512_checkerboard.mat
D

In [63]:
test_results = [str(p) for p in Path('tests/').rglob("MatrixC*")]

data = {'run':[], 'time':[], 'size':[], 'pattern':[], 'sparsity':[],
        'algorithm': [], 'gflops': [], 'rel_error': []}

for file in test_results:
    pattern = r'tests/MatrixC_(\d+)_([^_]+)_run(\d+)\.res'
    match = re.match(pattern, file)

    matrix_size = int(match.group(1))
    matrix_pattern = match.group(2)
    run_number = int(match.group(3))

    with open(file) as f:
        lines = f.readlines()

    for line in range(3, len(lines), 5):
        if lines[line].strip().startswith('CUDA'):
            line += 1
        algo = lines[line].split("time")[0].strip()
        time = float(lines[line].split(":")[1].strip())
        rel_error = float(lines[line+3].split(":")[1].strip())
        #(2 × N³ - N²) / (time_in_ms × 10⁶)

        if time != 0:
            gflops = (2.0 * matrix_size ** 3) / (time * 10 ** 6)
        else:
            gflops = -1

        data['run'].append(run_number)
        data['time'].append(time)
        data['size'].append(matrix_size)
        data['pattern'].append(matrix_pattern)

        data['sparsity'].append(0.5 if pattern == "checkerboard" else 16
                                                                      / matrix_size)
        data['algorithm'].append(algo)
        data['gflops'].append(gflops)
        data['rel_error'].append(rel_error)

df = pd.DataFrame(data)
df.loc[df['algorithm'] == 'Dense on GPU', 'rel_error'] = 0.0
df.loc[df['gflops'] == -1, 'gflops'] = np.nan
df[df['algorithm'] == 'Dense on GPU'].head()
df.to_json('results.json')
df.to_csv('results.csv')
#df.to_excel('results.xlsx')

In [42]:
df.columns

Index(['run', 'time', 'size', 'pattern', 'sparsity', 'algorithm', 'gflops',
       'rel_error'],
      dtype='object')

In [43]:
df.head()

,run,time,size,pattern,sparsity,algorithm,gflops,rel_error
0,8,57.0,2048,blockdiagonal,0.007812,Dense on GPU,301.401214,0.000000
1,8,4.0,2048,blockdiagonal,0.007812,Dense WMMA,4294.967296,0.000157
2,8,2.0,2048,blockdiagonal,0.007812,SpMM 1,8589.934592,0.000000
3,8,7.0,2048,blockdiagonal,0.007812,SpMM 2,2454.267026,0.000000
4,8,6.0,2048,blockdiagonal,0.007812,SpMM 3,2863.311531,0.000000


In [56]:
algorithms = df['algorithm'].unique().tolist()
algorithms

['Dense on GPU',
 'Dense WMMA',
 'SpMM 1',
 'SpMM 2',
 'SpMM 3',
 'SpMM with Tensors']

In [65]:
avg_datas = list()

for algorithm in algorithms:
    for size in matrix_sizes:
        group_df = df[(df['algorithm'] == algorithm) & (df['size'] == size) &
                      (df['pattern'] == 'checkerboard')]
        avg_dat = {'run': 'avg',
                   'time': group_df['time'].mean(),
                   'size': size,
                   'pattern': 'checkerboard',
                   'sparsity': group_df['sparsity'].iloc[0],
                   'algorithm': algorithm,
                   'gflops_mean': group_df['gflops'].mean(),
                   'gflops_med': group_df['gflops'].median(),
                   'gflops_min': group_df['gflops'].min(),
                   'gflops_max': group_df['gflops'].max(),
                   'gflops_var': group_df['gflops'].var(),
                   'rel_error_mean': group_df['rel_error'].mean(),
                   'rel_error_med': group_df['rel_error'].median(),
                   'rel_error_min': group_df['rel_error'].min(),
                   'rel_error_max': group_df['rel_error'].max(),
                   'rel_error_var': group_df['rel_error'].var(),
                   }
        avg_datas.append(avg_dat)

grouped_df = pd.DataFrame(avg_datas)
grouped_df.to_csv('avg_results.csv')

/home/huertasg/Dev/MatrixMultiplicationCUDA/.venv/lib/python3.12/site-packages/numpy/lib/_nanfunctions_impl.py:1214: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
/home/huertasg/Dev/MatrixMultiplicationCUDA/.venv/lib/python3.12/site-packages/numpy/lib/_nanfunctions_impl.py:1214: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
/home/huertasg/Dev/MatrixMultiplicationCUDA/.venv/lib/python3.12/site-packages/numpy/lib/_nanfunctions_impl.py:1214: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
/home/huertasg/Dev/MatrixMultiplicationCUDA/.venv/lib/python3.12/site-packages/numpy/lib/_nanfunctions_impl.py:1214: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)


In [59]:
grouped_df

,run,time,size,pattern,sparsity,algorithm,gflops_mean,gflops_med,gflops_min,gflops_max,rel_error_mean,rel_error_med,rel_error_min,rel_error_max
0,avg,1.000000,512,checkerboard,0.031250,Dense on GPU,268.435456,268.435456,268.435456,268.435456,0.000000,0.000000,0.000000,0.000000
1,avg,7.233333,1024,checkerboard,0.015625,Dense on GPU,297.835530,306.783378,268.435456,306.783378,0.000000,0.000000,0.000000,0.000000
2,avg,26.100000,1536,checkerboard,0.010417,Dense on GPU,277.752035,278.759897,258.848475,278.759897,0.000000,0.000000,0.000000,0.000000
3,avg,57.366667,2048,checkerboard,0.007812,Dense on GPU,299.513126,301.401214,286.331153,301.401214,0.000000,0.000000,0.000000,0.000000
4,avg,767.433333,4096,checkerboard,0.003906,Dense on GPU,179.093790,178.956971,177.340585,180.603093,0.000000,0.000000,0.000000,0.000000
5,avg,1077.566667,4608,checkerboard,0.003472,Dense on GPU,181.604671,181.698651,180.692011,182.375999,0.000000,0.000000,0.000000,0.000000
6,avg,1466.933333,5120,checkerboard,0.003125,Dense on GPU,182.991804,182.920264,182.361043,183.608383,0.000000,0.000000,0.000000,0.000000
7,avg,1939.766667,5632,checkerboard,0.002841,Dense on GPU,184.191809,184.311383,182.942955,184.644750,0.000000,0.000000,0.000000,0.000000
8,avg,2502.500000,6144,checkerboard,0.002604,Dense on GPU,185.357537,185.357237,184.950745,185.765506,0.000000,0.000000,0.000000,0.000000
9,avg,5869.466667,8192,checkerboard,0.001953,Dense on GPU,187.327426,187.326286,187.150915,187.501983,0.000000,0.000000,0.000000,0.000000
